# AWS API Gateway

<p align=center><a href=https://aws.amazon.com/api-gateway><img src=images/API_Gateway_Logo.png width=250></a></p>

## Introduction
> **Amazon API Gateway** is an AWS service that allows the creation, maintenance and securing of scalable **REST**, **HTTP** and **Websocket** APIs. APIs can be created to access AWS services or other web services and data stored in the AWS Cloud. As a developer, you can use the service to create APIs that serve your client's applications or that are available to third-party app developers.

## Types of APIs

* **REST APIs** are the de facto standard for client-server communication. Although REST APIs have a higher latency rate than HTTP APIs, they have full endpoint support, supporting edge-optimised, regional and private endpoints. They also support endpoint caching and response validation, unlike HTTP APIs.<br>
 
* **HTTP APIs** were created to be the version of REST APIs that offers relatively low latency rates and cost implementation. Amazon created this API in 2019 and reported that it can offer up to a 60% reduction in latency. However, it does not offer as many features as REST APIs, and it only has regional endpoint support on AWS.<br>
 
* **Websocket APIs** allow full-duplex communication between client and server. Simply put, communication can be initiated in both directions. The client can send requests to the server AND the server can send requests to the client. They can be used to create real-time applications, such as messaging applications, where the server might send you a notification, and you might send it a message for a friend.

The full breakdown of the differences between HTTP APIs and REST APIs by AWS can be found [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/http-api-vs-rest.html).



## Creating an AWS Gateway REST API

Upon opening the AWS API gateway from the AWS-services dashboard, you will be met with the landing page. Here, you can select the type of API you want to build (HTTP, REST or WebSocket) or import your externally developed API. For this example, we will be building a new REST API. Select build from the REST API option to create a new REST API. The goal is to create a REST API which integrates with a lambda function that facilitates machine-learning model predictions.

From the API-gateway dashboard, select REST API, the New API option. Name the API **ML_Model**, and provide a description. Subsequently, you will be required to select the Endpoint type for the API.

### API endpoint types

* **Regional:** For APIs that are only available in the specific AWS region for which the API was created.
* **Edge optimised:** For APIs to be deployed to the AWS Cloudfront.
* **Private:** For APIs that are only available through a configured VPC endpoint.

More details on API endpoints can be found in the AWS documentation [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-api-endpoint-types.html).

In our case, a regional API will suffice. Once selected, create the API by clicking the `Create API` button.


<img src="images/API_create.png?modified=12345678" />




After creating your API, you will be met with the main dashboard, where you can then begin to build the API. The sidebar on the left-hand side provides many options for creating your API. We will focus on the main areas:

* **Resources:** Collection of resources and methods, such as HTTP endpoints, lambda functions or AWS services, which can interact with the created API.
* **Stages:** Create staging areas for your gateway (e.g. pre-deployment and post-deployment areas) for testing the gateway before serving the API to the end-user.
* **Authorizers:** Use a lambda function or Amazon cognito user pool to control which users have access to your gateway.
* **Models:** Define the structure of the incoming payload between the client and server.
* **Gateway Responses:** Configure the common gateway responses, e.g. access denied, default 4XX, bad request body in the case of the gateway failing, etc.
* **Resource Policy:** Configure access control to the API with IAM conditions or VPC endpoints.
* **Documentation:** Add documentation so that users of your API can understand how it is used.<br></br>
  


<img src="images/API_sidebar.png?modified=12345678" />

Now that we have a general overview of the various options API gateway provides for building an API, we begin creating an API gateway that returns results from a lambda function. 

## Integrating Lambda Functions with the API Gateway

To familiarise ourselves with using the API-gateway interface and developing new methods, we create a simple lambda function and integrate it with our gateway. Head over to AWS Lambda, and create a new Lambda function named **API_test**. Add the following code to the function:


In [ ]:
import json

def lambda_handler(event, context):
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('Our first API gateway test was a success!!')
    }


Afterwards, head back to API, where we will create a method to test the integration with our lambda function. Go to the API you created, and select resources. Then, click the Actions dropdown list. You will be presented with the various options for creating resources and methods:

* **Create Method:** The methods are the same methods you would find in any other REST API, such as GET, POST, PUSH, etc.
* **Create Resource:** Define a new API resource that we can assign methods to operate on. 
* **Enable Cross-Origin Resource Sharing(CORS):** CORS sharing allows you to define how resources in another domain might interact securely with your API gateway. More details can be found [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/cors.html). <br></br>


<img src="images/API_methods_resources.png?modified=12345678" />

Now, we create a new method to obtain a response from our lambda function. Select actions, create method, and **GET** from the dropdown. To use a method, you need to 'integrate' it with an endpoint on the backend. These endpoints can be an integration, such as an HTTP endpoint, lambda function or AWS service. Below are the types of integrations available:

* **Lambda function:** Integrate your service with an AWS lambda function.
* **HTTP:** Enable your API to communicate with an HTTP-endpoint backend.
* **Mock:** Allow the API to return a response without sending the backend; useful for testing your API.
* **AWS Service:** Integrate your API with an AWS service.
* **VPC Link:** Integrate your API with a VPC link, allowing you to connect your API to private resources.

Since we are interacting with a lambda function, we will apply the lambda-function integration to our **GET** method. Detailed information on the types of integrations can be found in the AWS docs [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-api-integration-types.html).

### Proxy Integrations

When creating your integration, you will have the option of creating it as a proxy integration. Proxy integrations provide the selected integration access to many resources and features at once, without specifying multiple resource paths using the greedy parameter, **{proxy+}**. For instance, consider that your API has the resource paths:

> - **car/{type}/{subtype}/{parts}**
> - **car/{type}/{subtype}/{colour}**
> - **car/{type}/{subtype}/{colour}/{cost}**

As your API grows, it can be very tedious to specify these paths individually for your integration when you enable access to all available resources. By creating a proxy resource with the **{proxy+}** parameter and the **ANY** method, you can provide your integration with access to all available resources. A more detailed explanation of proxy integrations can be found [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/set-up-lambda-proxy-integrations.html). For our lambda function, we will not employ a proxy integration.

<img src="images/API_method_setup.png?modified=12345678" />

Once the integration is created and configured with the **GET** method, you will be met with the Method Execution screen, showing the execution order of the methods and allowing you to test the method. There are four main steps when processing the request and response. In our case, two request steps before the lambda function are called, followed by two response steps. 

* **Method Request:** This step is used to configure security settings (API keys, authorisation, validation), query-string parameters and request headers.
* **Integration Request:** At this step, the gateway makes a request to the integration, passing along the request data and transforming it, if necessary.
* **Integration Response:** The integration-response step is the result returned from the backend, in our case, the lambda function. Afterwards, the data are transformed if required and sent to the method response. 
* **Method Response:** The method response contains the output of the API, which will be served to the user. Usually, the output contains the HTTP status code, headers and a body.

You can read more about responses and request-data mapping [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/request-response-data-mappings.html). Click the test button, as well as the following test button on the next screen, and you will get the response from the method.


<img src="images/API_gateway_method_execution.png?modified=12345678" />

After clicking the test button, you will be presented with the test pane, which will display the results of the initiated method. In this case, we can see the expected result. The method returns a status 200, indicating success, as well as the expected body of text from our lambda function. Notice that the response also includes the **Response Header**. API headers contain additional metadata about the API call. They may contain information about authorisation, cookies, caching, etc. and are a good place to find insights into failing method calls.

<img src="images/API_test_pane.png?modified=12345678" />

## API Repsonses in Python

Now that we have created an API, we can use the Python requests library to test the API and obtain a response. Firstly, however, we must deploy the API so that we can request a response from it. To do that, click the "Actions" dropdown and select "Deploy API"

## Conclusion
At this point, we should have a good understanding of

* the different types of APIs available from API Gateway.
* the different types of methods we can create and the available integrations from API Gateway. 
* the difference between responses and requests.
* the difference between methods and resources.
* how to configure different types of API methods and the role of each method. 
* how to configure an AWS lambda function to integrate with APIs and test the method to return an API response.
